## Missed Trash Pickups

In this data question you will be working data of service request related to missed trash pickups from hubNashville, Metro Nashville government's comprehensive customer service system (https://hub.nashville.gov).

As part of Metro's contract with Red River Waste Solutions, failure to remedy an action or inaction will result in liquidated damages. One category of liquidated damages is related to chronic problems in any category of service at the same premises. A chronic problem is defined as more than one missed pickup for any address. The first missed pickup will not result in a fine; however, every subsequent missed pickup will result in a $200 fine.

Your job is to determine the total amount of damages due to missed pickups. Note that not all rows that you have been provided correspond to missed pickups and that you will need to ensure that you are only counting missed pickups.

After determining the total amount of damages, you can look at other questions:

* What other types of complaints are there?
* Are there any geospatial analysis you can do?  Which visualizations can you create?
* How do metro crews compare to the contractor's performance?
* How much does each trash hauler owe?
* What were to total missed pickup by route?

In [1]:
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import branca

In [2]:
tp = gpd.read_file('../data/trash_hauler_report_with_lat_lng.csv')
tp.head()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,geometry
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207,RED RIVER,3205,2,1727970.412,686779.4781,-86.81539226,36.21729166,None
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1,1721259.366,685444.7996,-86.83810278,36.21346977,None
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20,1707026.753,659887.4716,-86.8855624,36.14292283,None
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,1735691.771,685027.2459,-86.7891702,36.21265241,None
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,1710185.772,664205.1011,-86.87499468,36.15486093,None


In [3]:
tp.tail()

,Request Number,Date Opened,Request,Description,Incident Address,Zip Code,Trash Hauler,Trash Route,Council District,State Plan X,State Plan Y,LONGITUDE,LATITUDE,geometry
20221,267125,11/1/2019,Trash - Curbside/Alley Missed Pickup,MISSED...NEIGHBORS MISSED,2731 Murfreesboro Pike,37013,RED RIVER,4502,32,1781137.263,632448.5511,-86.63397,36.06913,None
20222,267126,11/1/2019,Trash - Curbside/Alley Missed Pickup,entire alley,"1621 Long Ave, Nashville, TN 37206, United States",37206,METRO,9508,6,1749711.399,669201.6016,-86.74124224,36.16948232,None
20223,267130,11/1/2019,Trash - Curbside/Alley Missed Pickup,missed several,"2943 Windemere Cir, Nashville, TN 37214, Unite...",37214,RED RIVER,1502,15,1770293.388,674936.3038,-86.67164737,36.18564312,None
20224,267134,11/1/2019,Trash - Curbside/Alley Missed Pickup,Caller stated trash was missed & were only pic...,"3325 Murfreesboro Pike, Nashville, TN 37013, U...",37013,RED RIVER,4502,32,1785224.998,627146.4002,-86.62002525,36.05463692,None
20225,267137,11/1/2019,Trash - Curbside/Alley Missed Pickup,possibly others missed as well,"604 Somerset Ct, Nashville, TN 37217, United S...",37217,RED RIVER,2505,29,1781360.323,637742.0068,-86.63333117,36.08367504,None


In [4]:
tp.shape

(20226, 14)

Reshaping and Renaming Dataframe

In [5]:
tp = tp.drop(columns = ['geometry'])

In [6]:
tp = tp.rename(columns = {
    "Request Number" : "request_num",
    "Date Opened" : "date_opened",
    "Request" : 'request',
    "Description" : "desc",
    "Incident Address" : "address",
    "Zip Code" : "zip",
    "Trash Hauler" : 'truck',
    "Trash Route" : 'route',
    "Council District" : 'cdistrict',
    "State Plan X" : "spx",
    "State Plan Y" : "spy",
    "LATITUDE" : "lat",
    "LONGITUDE" : "lon"})
tp.head()

,request_num,date_opened,request,desc,address,zip,truck,route,cdistrict,spx,spy,lon,lat
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207,RED RIVER,3205,2,1727970.412,686779.4781,-86.81539226,36.21729166
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1,1721259.366,685444.7996,-86.83810278,36.21346977
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20,1707026.753,659887.4716,-86.8855624,36.14292283
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,1735691.771,685027.2459,-86.7891702,36.21265241
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,1710185.772,664205.1011,-86.87499468,36.15486093


Filter by red river truck only!

Checking if column 'request' contains substring 'Miss' AND creating new boolean column from results

In [7]:
tp['miss'] = (tp['request'].str.contains('Miss').map({True : 'yes', False : 'no'}))

In [8]:
tp.head()

,request_num,date_opened,request,desc,address,zip,truck,route,cdistrict,spx,spy,lon,lat,miss
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207,RED RIVER,3205,2,1727970.412,686779.4781,-86.81539226,36.21729166,no
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1,1721259.366,685444.7996,-86.83810278,36.21346977,yes
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20,1707026.753,659887.4716,-86.8855624,36.14292283,yes
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,1735691.771,685027.2459,-86.7891702,36.21265241,yes
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,1710185.772,664205.1011,-86.87499468,36.15486093,yes


Checking count of what rows contain substring "Miss" in request column (denoted by a 'yes' or 'no' in 'miss' column)

In [9]:
tp['miss'].value_counts()

miss
yes    15028
no      5198
Name: count, dtype: int64

Checking what other possible requests there are

In [10]:
tp.loc[tp['miss'].isin(['no'])].head()

,request_num,date_opened,request,desc,address,zip,truck,route,cdistrict,spx,spy,lon,lat,miss
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207,RED RIVER,3205,2,1727970.412,686779.4781,-86.81539226,36.21729166,no
5,25317,11/1/2017,Trash Collection Complaint,left trash cart in middle of driveway instead ...,3602 floral dr,37211,RED RIVER,4304,16,1751660.164,643215.2011,-86.73397993,36.09813983,no
6,25325,11/1/2017,Damage to Property,Trash/emptied Wednesday & now metal black-mail...,4721 Chalmers Dr,37215,RED RIVER,3303,34,1726300.965,637078.4022,-86.81964037,36.0807294,no
7,25327,11/1/2017,Trash Collection Complaint,"Trash out on time, miss again Tuesday. ALLEY",1816 Jo Johnston Ave,37203,METRO,9208,21,1731459.367,666013.6012,-86.80298771,36.16032965,no
11,25350,11/1/2017,Trash Collection Complaint,Missed- 4th week in a row.,3210 Whites Creek Pike,37207,RED RIVER,3201,3,1726432.966,691919.2003,-86.82075005,36.23137438,no


In [11]:
tp.loc[tp['miss'].isin(['no'])].shape

(5198, 14)

I am not to include rows that don't denote a missed pickup in the request column but include information about a missed pickup or multiple missed pickups in the description.

Creating a new dataframe only including rows where requests specify a missed pickup.

In [12]:
tp['request'].unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint', 'Damage to Property'], dtype=object)

In [13]:
missed_pickups = tp.loc[tp['request'].str.contains('Trash - Curbside/Alley Missed Pickup')]
missed_pickups.head()

,request_num,date_opened,request,desc,address,zip,truck,route,cdistrict,spx,spy,lon,lat,miss
1,25274,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/Trash miss Tuesday.,4028 Clarksville Pike,37218,RED RIVER,4202,1,1721259.366,685444.7996,-86.83810278,36.21346977,yes
2,25276,11/1/2017,Trash - Curbside/Alley Missed Pickup,Curb/trash miss Tuesday.,6528 Thunderbird Dr,37209,RED RIVER,4205,20,1707026.753,659887.4716,-86.8855624,36.14292283,yes
3,25307,11/1/2017,Trash - Curbside/Alley Missed Pickup,missed,2603 old matthews rd,37207,WASTE IND,2206,2,1735691.771,685027.2459,-86.7891702,36.21265241,yes
4,25312,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed the even side of the road.,604 croley dr,37209,RED RIVER,4203,20,1710185.772,664205.1011,-86.87499468,36.15486093,yes
8,25330,11/1/2017,Trash - Curbside/Alley Missed Pickup,Missed.,4484 Lavergne Couchville Pike,37013,RED RIVER,4210,33,1794533.514,618749.3427,-86.58836424,36.03172793,yes


In [14]:
missed_pickups = missed_pickups.loc[missed_pickups['truck'] == 'RED RIVER']

Now I'm going to try and find which addresses appear multiple times in the missed pickups dataframe.

In [15]:
missed_pickups['address'].nunique()

8191

In [16]:
missed_pickups['address'].value_counts()

address
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
5135 Hickory Hollow Pkwy                                      16
802 Crescent Rd, Nashville, TN 37205, United States           16
607 Estes Rd, Nashville, TN 37215, United States              15
6007 Obrien Ave, Nashville, TN 37209, United States           14
                                                              ..
1968 Wild Oaks Ct, Nashville, TN 37013, United States          1
1070 Rural Hill Rd, Nashville, TN 37013, United States         1
100 Graeme Dr, Nashville, TN 37214, United States              1
2917 Leatherwood Dr, Nashville, TN 37214, United States        1
604 Somerset Ct, Nashville, TN 37217, United States            1
Name: count, Length: 8191, dtype: int64

In [17]:
fines = missed_pickups['address'].value_counts()
fines.head()

address
12546 Old Hickory Blvd, Nashville, TN 37013, United States    19
5135 Hickory Hollow Pkwy                                      16
802 Crescent Rd, Nashville, TN 37205, United States           16
607 Estes Rd, Nashville, TN 37215, United States              15
6007 Obrien Ave, Nashville, TN 37209, United States           14
Name: count, dtype: int64

In [18]:
fines = fines.reset_index()

In [19]:
fines.loc[fines['count'] > 1]

,address,count
0,"12546 Old Hickory Blvd, Nashville, TN 37013, U...",19
1,5135 Hickory Hollow Pkwy,16
2,"802 Crescent Rd, Nashville, TN 37205, United S...",16
3,"607 Estes Rd, Nashville, TN 37215, United States",15
4,"6007 Obrien Ave, Nashville, TN 37209, United S...",14
...,...,...
1546,"3508 Melva Dr, Nashville, TN 37211, United States",2
1547,3570 GONDOLA DR,2
1548,"3209 Jonesboro Dr, Nashville, TN 37214, United...",2
1549,4256 moss rd,2


For loop to calculate fines VVV

In [20]:
total_fines=0
for row_index, row_values in fines.iterrows():
    if row_values['count'] > 1:
        fine = (row_values['count']-1)*200
        total_fines = total_fines + fine

In [21]:
print(total_fines)

504800


## Total Fines: $504,800

double checking.....

In [22]:
total_missed = fines['count'].sum()

In [23]:
total_houses_missed = fines['address'].count()

In [24]:
total_houses_missed_over1 = 0
for row_index, row_values in fines.iterrows():
    if row_values['count'] > 1:
        total_houses_missed_over1 = total_houses_missed_over1 + 1

In [25]:
total_houses_missed_over1

1551

In [26]:
total_houses_missed

8191

In [27]:
total_houses_missed_over1/total_houses_missed

0.18935416921010867

In [28]:
total_fines/(total_missed*200)

0.23555762949136724

struggling...... :(

## Extra stuff, geospatial....

Are there any geospatial analysis you can do?  Which visualizations can you create?

In [29]:
tp.head(1)

,request_num,date_opened,request,desc,address,zip,truck,route,cdistrict,spx,spy,lon,lat,miss
0,25270,11/1/2017,Trash - Backdoor,"house with the wheel chair ramp, they share dr...",3817 Crouch Dr,37207,RED RIVER,3205,2,1727970.412,686779.4781,-86.81539226,36.21729166,no


In [30]:
tp.truck.unique()

array(['RED RIVER', 'WASTE IND', 'METRO', '', 'Metro'], dtype=object)

In [31]:
tp.request.unique()

array(['Trash - Backdoor', 'Trash - Curbside/Alley Missed Pickup',
       'Trash Collection Complaint', 'Damage to Property'], dtype=object)

In [32]:
tp['lat'] = pd.to_numeric(tp['lat'])
tp['lon'] = pd.to_numeric(tp['lon'])

In [33]:
m = folium.Map(location=[36.174465, -86.767960], zoom_start=10)

In [40]:
marker_cluster = MarkerCluster().add_to(m)

for row_index, row_values in tp.iterrows():
    if pd.isna(row_values['lat']) or pd.isna(row_values['lon']):
        continue
    
    if row_values['truck'] in (['METRO','metro','Metro']):
        markercolor= "blue"
    elif row_values['truck'] in (['WASTE IND']):
        markercolor= "green"
    elif row_values['truck'] in (['RED RIVER']):
        markercolor= "red"
    else:
        markercolor = "black"
    
    loc = [row_values['lat'], row_values['lon']]
    address = str(row_values['address'])
    description = str(row_values['desc'])
    date = str(row_values['date_opened'])
    request_num = str(row_values['request_num'])
    comp = str(row_values['truck'])
    req = str(row_values['request'])
    icon=folium.Icon(color=markercolor,icon="house", prefix='fa')

    html = """<h3>"""+address+"""</h3><br>
    <p> Opened: """+date+"""<br><br>
    Contractor: """+comp+"""<br><br>
    Request:"""+req+"""<br>
    Description: """+description+"""<br></p>"""

    iframe = branca.element.IFrame(html, width=250, height=200)
    popup = folium.Popup(iframe, max_width=500)
    
    marker = folium.Marker(
        location = loc, 
        popup = popup, 
        icon = icon) 

    marker.add_to(marker_cluster)

TemplateSyntaxError: Missing end of comment tag